# Oncoming

Here we describe some new features of the developing version of MolDrug

Let's install the developing version of MolDrug. I am assuming that you already have installed RDKit and Vina dependencies. For more details in how to install MolDrug visit [the installation instructions](https://moldrug.readthedocs.io/en/latest/source/installation.html).

In [ ]:
#! pip install git+https://github.com/ale94mleon/MolDrug.git@main